In [ ]:
from services.score import Score
from services import resume_scraper
from db_tools import get_job_desc

resume_path = 'data\\resumes\\Jackson_Giemza.pdf'
# raw_resume = resume_scraper.extract_text_from_pdf(resume_path)
raw_resume = resume_scraper.extract_text_from_pdf('data\\resumes\\EduardoBenjaminResume-1Page-3.pdf')
raw_jd = get_job_desc('1325')

scorer = Score(raw_resume, raw_jd)
print(scorer.compute_similarity())
print()

tensor([[0.0193]])
* This is an in\-office position.




**General Statement of Duties**


This exempt position is responsible for ensuring the cybersecurity and physical security of the Parker Water and Sanitation District (PWSD) IT/OT and infrastructure assets. This position involves participating in the Disaster Recovery program, Business Continuity planning, vulnerability assessments, and penetration testing. This position manages vendor relationships, Zero Trust Architecture, security information and event management, as well as performing root cause analysis, and developing training tools to prevent/mitigate incidents. The Cybersecurity Analyst establishes and maintains key performance indicators (KPIs) to measure the effectiveness of cybersecurity efforts, audit and enforce patch cadence and security policies, and adopts industry frameworks and best practices as a critical component of continuous improvement, training, and education for PWSD staff in physical and cybersecurity b

In [14]:
import re
from sentence_transformers import SentenceTransformer
import string
import nltk
# nltk.download('stopwords')
# nltk.download('punkt_tab')
# nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

class Score:
    def __init__(self, raw_resume, raw_jd):
        self.raw_resume = raw_resume
        self.raw_jd = raw_jd

        self.set_of_stopwords = set(stopwords.words("english") + list(string.punctuation))

        self.lemmatizer = WordNetLemmatizer()

    def clean_text(self, raw_text):        

        # Convert text to lowercase and tokenize into words
        tokens = word_tokenize(raw_text.lower())
        # Remove stopwords and punctuation
        tokens = [token for token in tokens if token not in self.set_of_stopwords]
        # Lemmatize the remaining words
        tokens = [self.lemmatizer.lemmatize(token) for token in tokens]
        # Join the tokens back into a single string
        cleaned_text = " ".join(tokens)
        return cleaned_text

    def extract_section(self, section_name, next_sections):
        """
        Extracts a section of the resume based on section headers.

        :param section_name: The name of the section to extract.
        :param next_sections: Possible names of the next sections.
        :return: Extracted text of the section.
        """
        pattern = re.compile(rf'{section_name}\s*[:\n]', re.IGNORECASE)
        match = pattern.search(self.raw_resume)

        if not match:
            return ""

        start = match.end()
        end = len(self.raw_resume)

        for next_section in next_sections:
            next_match = re.search(rf'{next_section}\s*[:\n]', self.raw_resume[start:], re.IGNORECASE)
            if next_match:
                end = start + next_match.start()
                break

        section_text = self.raw_resume[start:end].strip()
        return section_text

    def clean_resume(self):
        # Possible section names for skills and experience
        experience_sections = [
            "Work Experience", "Professional Experience", "Employment History",
            "Internship Experience", "Volunteer Experience"
        ]
        skills_sections = ["Skills", "Technical Skills", "Software Skills"]

        next_sections = [
            "Projects", "Certifications", "Licenses", "Awards", "Honors",
            "Publications", "References", "Education", "Objective", "Summary"
        ]

        # Extract sections
        experience_text = ""
        for exp_section in experience_sections:
            experience_text = self.extract_section(exp_section, next_sections)
            if experience_text:
                break

        skills_text = ""
        for skill_section in skills_sections:
            skills_text = self.extract_section(skill_section, next_sections)
            if skills_text:
                break

        # Clean extracted sections
        cleaned_experience = self.clean_text(experience_text) if experience_text else ""
        cleaned_skills = self.clean_text(skills_text) if skills_text else ""

        return cleaned_experience + " " + cleaned_skills

    def compute_similarity(self):
        cleaned_resume = self.clean_resume()
        cleaned_jd = self.clean_text(self.raw_jd)

        model = SentenceTransformer('all-MiniLM-L6-v2')
        sentences = [cleaned_resume, cleaned_jd]
        embeddings1 = model.encode(sentences[0])
        embeddings2 = model.encode(sentences[1])
        
        similarity_score = model.similarity(embeddings1, embeddings2)

        return similarity_score

In [23]:
resume_path = 'data\\resumes\\Jackson_Giemza.pdf'
raw_resume = resume_scraper.extract_text_from_pdf(resume_path)
# raw_resume = resume_scraper.extract_text_from_pdf('data\\resumes\\EduardoBenjaminResume-1Page-3.pdf')

In [24]:
scorer = Score(raw_resume, raw_jd)

In [25]:
scorer.compute_similarity()

tensor([[0.2394]])

In [19]:
print(raw_resume)

Eduardo Benjamin

Boulder, Co - eddiebenjamin86@gmail.com - eduardobenjamin.site

OBJECTIVE

Data-driven analyst with expertise in predictive modeling, data engineering, and process automation. Successfully
streamlined loan data processing for risk assessments, enhancing reporting efficiency and supporting strategic decision-
making.

EDUCATION

University Of Colorado Boulder | Bachelor of Information Science | 2022 - 2025 | 3.8 GPA

Arapahoe Community College | Associates of Science | 2019 - 2022

WORK EXPERIENCE
Data Engineer/Business Analyst Internship | SitusAMC
Risk Advisory Department | May - August 2024
e Analyzed Large-Scale Loan Datasets: Managed and cleaned 20,000+ loan records, improving
data accuracy and optimizing reporting efficiency using SQL and Python.
- Developed SQL-Based Data Processing Tools: Engineered a parser to standardize internal SQL
dialects, streamlining data transformation and reducing manual work.
- Built Data Pipelines for Loan Processing: Designed SQL a